In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
data = spark.read.csv('/FileStore/tables/seeds_dataset.csv', header = True, inferSchema = True)

In [4]:
data.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|
14.88| 14.57| 0.8811| 5.553999999999999| 3.333| 1.018| 4.956|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|
13.84| 13.94| 0.8955| 5.324|3.3789999999999996| 2.259| 4.805|
16.14| 14.99| 0.9034|5.6579999999999995| 3.562| 1.355| 5.175|
14.38| 14.21| 0.8951| 5.386| 3.312| 2.4619999999999997| 4.956|
14.69| 14.49| 0.8799| 5.563| 3.259| 3.5860000000000003| 5.218999999999999|
14.11| 14.1| 0.8911| 5.42| 3.302| 2.7| 5.0|
16.63| 15.46| 0.8747| 6.053| 3.465| 2.04| 5.877000000000001|
16.44| 15.25| 0.888|5.8839999999999995| 3.505| 1.969|5.5329999999999995|
15.26| 14.85| 0.8696|5.7139999999999995| 3.242| 4.543| 5.314|
14.03| 14.16| 0.8796| 5.438| 3.201| 1.7169999999999999| 5.001|
13.89| 14.02| 0.888| 5.439| 3.199| 3.986| 4.738|
13.78| 14.06| 0.8759| 5.479| 3.156| 3.136| 4.872|
13.74| 14.05| 0.8744| 5.482| 3.114| 2.932| 4.825|
14.59| 14.28| 0.8993| 5.351| 3.333| 4.185| 4.781000000000001|
13.99| 13.83| 0.9183| 5.119| 3.383| 5.234| 4.781000000000001|
15.69| 14.75| 0.9058| 5.527| 3.514| 1.599| 5.046|
 14.7| 14.21| 0.9153| 5.205| 3.466| 1.767| 4.649|
12.72| 13.57| 0.8686| 5.226| 3.049| 4.102| 4.914|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
only showing top 20 rows

In [5]:
from pyspark.ml.clustering import KMeans

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

Out[ 10 ]: 
['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [8]:
assembler = VectorAssembler(inputCols= data.columns,
                           outputCol ='features')

In [9]:
final_data = assembler.transform(data)

In [10]:
from pyspark.ml.feature import StandardScaler

In [11]:
scaler = StandardScaler(inputCol= 'features',
                       outputCol = 'scaled' )

In [12]:
final_data = scaler.fit(final_data).transform(final_data)

In [13]:
Kmeans = KMeans(featuresCol= 'scaled', k = 3 )

In [14]:
model = Kmeans.fit(final_data)

In [15]:
wssse = model.computeCost(final_data)
wssse

Out[ 24 ]: 429.07559671506715

In [16]:
centers = model.clusterCenters()
centers

Out[ 25 ]: 
[array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062,
 9.748067 , 2.39849968, 12.2661748 ]),
 array([ 4.87257659, 10.88120146, 37.27692543, 12.3410157 ,
 8.55443412, 1.81649011, 10.32998598]),
 array([ 4.06105916, 10.13979506, 35.80536984, 11.82133095,
 7.50395937, 3.27184732, 10.42126018])]

In [17]:
model.transform(final_data).select('prediction').show()

+----------+
prediction|
+----------+
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 0|
 0|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 2|
+----------+
only showing top 20 rows